## Spatial Pooler（空間プーリング）

In [1]:
from htm.bindings.sdr import SDR
from htm.algorithms import SpatialPooler as SP

In [3]:
from htm.bindings.encoders import ScalarEncoder, ScalarEncoderParameters
categories = ("unknown","cat", "dog", "monkey", "slow loris")

scalarEncoderParams = ScalarEncoderParameters()
scalarEncoderParams.minimum = 1
scalarEncoderParams.maximum = 5
scalarEncoderParams.activeBits = 3
scalarEncoderParams.category = True

enc = ScalarEncoder(scalarEncoderParams)

cat = enc.encode(categories.index("cat")+1)
dog = enc.encode(categories.index("dog")+1)
monkey = enc.encode(categories.index("monkey")+1)
loris = enc.encode(categories.index("slow loris")+1)

print ("cat =       ", cat)
print ("dog =       ", dog)
print ("monkey =    ", monkey)
print ("slow loris =", loris)

cat =        SDR( 15 ) 3, 4, 5
dog =        SDR( 15 ) 6, 7, 8
monkey =     SDR( 15 ) 9, 10, 11
slow loris = SDR( 15 ) 12, 13, 14


In [5]:
print (len(cat.dense))
print (cat.dense)

15
[0 0 0 1 1 1 0 0 0 0 0 0 0 0 0]


**`SpatialPooler(...)`**

空間プーラーを実装したクラスである。領域のカラムと入力ビットの関係を扱う。

この関数の主要なパブリック・インタフェースは "compute" メソッドであり、入力ベクトルを受け取り、activeColumns カラムのリストを返す。

```
sp = SpatialPooler(...)
for line in file:
  inputVector = numpy.array(line)
  sp.compute(inputVector)
  ...
```

**パラメータ**
- `inputDimensions` - (int) 入力ベクトルの次元を表すシーケンス
- `columnDimensions` - (int) リージョン内のカラムの寸法を表すシーケンス
- `potentialRadius` - (int) 各カラムが潜在的に接続できる入力の範囲を決定します
- `numActiveColumnsPerInhArea` - (float) アクティブなカラムの密度を制御する別の方法
- `globalInhibition` - (bool) true の場合、抑制フェーズの間、勝利したカラムは、リージョン全体の中で最もアクティブなカラムとして選択されます
- `synPermActiveInc` - (float) アクティブなシナプスが各ラウンドで増加する量
- `potentialPct` - (float) カラムのポテンシャル半径内で、カラムが接続できる入力のパーセンテージ

**`getConnectedSynapses(columnIndex, connectedSynapses)`**

**パラメータ**
- connectedSynapses -  (list)は上書きされます。

**戻り値**
- (iter) 指定されたカラムに接続されているシナプスの数を返す。

In [56]:
inputSDR  = SDR( dimensions = (4,4) )
outputSDR = SDR( dimensions = (8,8) )

sp = SP(inputSDR.dimensions,
        outputSDR.dimensions,
        potentialRadius = int(0.5 * inputSDR.size),
        localAreaDensity = .02,
        globalInhibition = True,
        seed = 0,
        synPermActiveInc = 0.01,
        potentialPct = 0.85)

import numpy
#for column in range(4):
#    connected = numpy.zeros((15,), dtype="int")
#    sp.getConnectedSynapses(column, connected)
 #   print (connected)

**`compute(inputVector, learn, activeArray)`**

これは SpatialPooler クラスの主要なパブリック・メソッドです。
この関数は入力ベクトルを受け取り、アクティブなカラムのインデックスを出力します。
`learn` が True に設定されている場合、このメソッドはカラムの永続性も更新します。

**パラメーター**

- `inputVector` - 空間プーラーへの入力を構成する0と1のnumpy配列．配列は1次元配列として扱われるので，配列の次元はクラスのコンストラクタで指定された次元と正確に一致する必要はありません．

- `learn` - 学習を行うかどうかを示すブール値．学習はシナプスの永続値を更新し、モデルの「状態」を修正することを意味する

- `activeArray` - カラム数と同じサイズの配列。関数が返す前に、この配列にはアクティブなカラムのインデックスに 1 が、それ以外の場所には 0 が格納されます。

In [53]:
#output = numpy.zeros((4,), dtype="int")
#sp.compute(cat, learn=True, activeArray=output)
sp.compute(cat, True, outputSDR)
print (outputSDR)

RuntimeError: Exception: Sdr.cpp(158) message: CHECK FAILED: "newSize == size" SDR.reshape changed the size of the SDR!

`xrange(20)`20回学習

In [44]:
for _ in range(20):
    sp.compute(cat, True, outputSDR)
print (outputSDR)

#for column in xrange(4):
#    connected = numpy.zeros((15,), dtype="int")
#    sp.getConnectedSynapses(column, connected)
#    print connected

SDR( 52, 1 ) 0


`xrange(200)`200回学習

In [46]:
for _ in range(200):
    sp.compute(cat, True, outputSDR)
    sp.compute(dog, True, outputSDR)
    sp.compute(monkey, True, outputSDR)
    sp.compute(loris, True, outputSDR)
print (outputSDR)

SDR( 52, 1 ) 3


In [ ]:
for column in xrange(4):
    connected = numpy.zeros((15,), dtype="int")
    sp.getConnectedSynapses(column, connected)
    print connected

In [ ]:
noisyCat = numpy.zeros((15,), dtype="uint32")
noisyCat[3] = 1
noisyCat[4] = 1
# This is part of dog!
noisyCat[6] = 1
print noisyCat

In [ ]:
sp.compute(noisyCat, learn=False, activeArray=output)
print output  # matches cat!